In this demo we will go over the basics of the Ray Job Submission Client in the SDK

In [ ]:
from codeflare_sdk import Cluster, ClusterConfiguration, TokenAuthentication

In [ ]:
# Create authentication object for user permissions
# IF unused, SDK will automatically check for default kubeconfig, then in-cluster config
# KubeConfigFileAuthentication can also be used to specify kubeconfig path manually
auth = TokenAuthentication(
    token="REPLACE_ME",
    server="REPLACE_ME",
    skip_tls=True
)
auth.login()

In [ ]:
# Create and configure our cluster object
# The SDK will try to find the name of your default local queue based on the annotation "kueue.x-k8s.io/default-queue": "true" unless you specify the local queue manually below
cluster = Cluster(ClusterConfiguration(
    name='jobtest',
    head_gpus=0,  # For GPU enabled workloads set the head_gpus and num_gpus
    num_gpus=0,
    num_workers=2,
    min_cpus=1,
    max_cpus=1,
    min_memory=4,
    max_memory=4,
    image="quay.io/project-codeflare/ray:latest-py39-cu118",
    write_to_file=False,  # When enabled Ray Cluster yaml files are written to /HOME/.codeflare/resources 
    # local_queue="local-queue-name" # Specify the local queue manually
))

In [ ]:
# Bring up the cluster
cluster.up()
cluster.wait_ready()

In [ ]:
cluster.details()

### Ray Job Submission

* Initialise the Cluster Job Client 
* Provide an entrypoint command directed to your job script
* Set up your runtime environment

In [ ]:
# Initialize the Job Submission Client
"""
The SDK will automatically gather the dashboard address and authenticate using the Ray Job Submission Client
"""
client = cluster.job_client

In [ ]:
# Submit an example mnist job using the Job Submission Client
submission_id = client.submit_job(
    entrypoint="python mnist_fashion.py",
    runtime_env={"working_dir": "./", "pip": "requirements.txt"},
)
print(submission_id)

In [ ]:
# Get the job's logs
client.get_job_logs(submission_id)

In [ ]:
# Get the job's status
client.get_job_status(submission_id)

In [ ]:
# Get job related info
client.get_job_info(submission_id)

In [ ]:
# List all existing jobs
client.list_jobs()

In [ ]:
# Iterate through the logs of a job 
async for lines in client.tail_job_logs(submission_id):
    print(lines, end="") 

In [ ]:
# Delete a job
# Can run client.stop_job(submission_id) first if job is still running
client.delete_job(submission_id)

In [ ]:
cluster.down()

In [ ]:
auth.logout()